In [1]:
import h5py
import numpy as np
# import matplotlib.pyplot as plt
from tifffile import imwrite, imread, tiffcomment, TiffFile
from ome_types import from_tiff
from glob import glob
import os
import affine_matrices as affine
from scipy.ndimage import affine_transform
from timeit import default_timer as timer
from npy2bdv import BdvWriter
from napari import Viewer
import lxml
from datetime import datetime

In [2]:
# root_dir = "A:/leo/micromanager/bonemarrow/data/"
# root_dir = "A:/leo/micromanager/dan/D_and_E/data/"
# root_dir = "A:/leo/micromanager/dan/D_and_E_frome4/data/"
root_dir = "E:/dopm_reslicing/"

# tiff_data_dir = os.path.join(root_dir, "y_stage_scan_20241216-171205_c0p0/")
tiff_data_dir = os.path.join(root_dir, "y_stage_scan_20241216-171205_c0p0")
# tiff_data_dir = os.path.join(root_dir, "y_stage_scan_20241209-144142")
# tiff_data_dir = os.path.join(root_dir, "y_stage_scan_20250116-160803")
# tiff_data_dir = os.path.join(root_dir, "y_stage_scan_20250116-172759")

folder_name = "HamamatsuHam_DCAM-dopm"

OPM_ANGLE = 45  # opm angle in degrees

tiff_dirs = glob(os.path.join(tiff_data_dir, "dOPM*"))  # unique dirs for each MDA dimension e.g. channel, time, z
print(tiff_dirs)

# what to do with z slices
only_keep_z0 = True

# group dirs with the same position pattern i.e. p0000, p0001, p0002
grouped_dirs = {}
grouped_details = {}

for tiff_dir in tiff_dirs:
    print(tiff_dir)
    view_details = os.path.basename(tiff_dir).split("_")
    angle = -OPM_ANGLE if "view1" in view_details[-1] else OPM_ANGLE
    position_key = view_details[2]
    position_str = position_key[1:]  # remove 'p' from 'p0000'
    time_str = view_details[1][1:]  # remove 't' from 't0000'
    z_str = view_details[3][1:]  # remove 'z' from 'z0000'
    channel_str = view_details[4][1:]  # remove 'c' from 'c0000'

    tiff_dir_dict = {}
    tiff_dir_dict["time"] = time_str
    tiff_dir_dict["z"] = z_str
    tiff_dir_dict["channel"] = channel_str
    tiff_dir_dict["dir"] = tiff_dir
    tiff_dir_dict["angle"] = angle

    if only_keep_z0 and int(z_str)>0:
        continue

    if position_key in grouped_dirs:
        grouped_dirs[position_key].append(tiff_dir)
        grouped_details[position_key].append(tiff_dir_dict)
    else:
        grouped_dirs[position_key] = [tiff_dir]
        grouped_details[position_key] = [tiff_dir_dict]
    
print(grouped_dirs)
print(grouped_details)
# unique_positions = list(grouped_dirs.keys())

for p, d in grouped_details.items():
    # number of channels in position (extract only 'channel' item from each dict)
    channels = list(set([int(i["channel"]) for i in d]))
    # do previous line in a neater way
    

    print(channels)

# now do the same thing for time and z
# then iterate over all combinations of positions, times and z to get the full list of files to process



['E:/dopm_reslicing/y_stage_scan_20241216-171205_c0p0\\dOPM_t0000_p0000_z0000_c0000_view1', 'E:/dopm_reslicing/y_stage_scan_20241216-171205_c0p0\\dOPM_t0000_p0000_z0000_c0000_view2']
E:/dopm_reslicing/y_stage_scan_20241216-171205_c0p0\dOPM_t0000_p0000_z0000_c0000_view1
E:/dopm_reslicing/y_stage_scan_20241216-171205_c0p0\dOPM_t0000_p0000_z0000_c0000_view2
{'p0000': ['E:/dopm_reslicing/y_stage_scan_20241216-171205_c0p0\\dOPM_t0000_p0000_z0000_c0000_view1', 'E:/dopm_reslicing/y_stage_scan_20241216-171205_c0p0\\dOPM_t0000_p0000_z0000_c0000_view2']}
{'p0000': [{'time': '0000', 'z': '0000', 'channel': '0000', 'dir': 'E:/dopm_reslicing/y_stage_scan_20241216-171205_c0p0\\dOPM_t0000_p0000_z0000_c0000_view1', 'angle': -45}, {'time': '0000', 'z': '0000', 'channel': '0000', 'dir': 'E:/dopm_reslicing/y_stage_scan_20241216-171205_c0p0\\dOPM_t0000_p0000_z0000_c0000_view2', 'angle': 45}]}
[0]


In [ ]:
# get current date for file timestamp
now = datetime.now()
date_time = now.strftime("%Y%m%d-%H%M%S")
print(date_time)

out_dir = os.path.join(root_dir, f"deskewed_{date_time}/")

if os.path.exists(out_dir):
    print("Output directory already exists, exiting")
    raise(FileExistsError)

# npos to do
stop_early = False
npos_max = 5

i_p = 0
# loop over grouped dirs to debug
for position, details in grouped_details.items():
    if stop_early and i_p>npos_max:
        break
    i_p+=1
    dirs = [d["dir"] for d in details]
    channels_strs = [d["channel"] for d in details]
    time_strs = [d["time"] for d in details]
    z_strs = [d["z"] for d in details]
    angles = [d["angle"] + OPM_ANGLE for d in details]
    angle_idxc = [0 if a == 0 else 1 for a in angles]

    # print all those dimenisons e.g. time, z, channel in a single print statement
    print("channels", channels_strs, "time", time_strs, "z", z_strs, "angle", angles)

    # to get n channels etc. use set(channels) etc. to get unique values
    n_channels = len(set(channels_strs))
    n_times = len(set(time_str))
    n_z = len(set(z_str)) # if not only_keep_z0 else 1
    n_angles = len(set(angles))

    print("n_channels", n_channels, "n_times", n_times, "n_z", n_z, "n_angles", n_angles)

    # for BDV dataset attributes
    channel_attr = [None]*n_channels
    angle_attr = [None]*n_angles

    # create BDV dataset
    save_path = os.path.join(out_dir, "pos_"+position)
    os.makedirs(save_path, exist_ok=True)
    save_file = os.path.join(save_path, "dataset.h5")

    bdv_writer = BdvWriter(save_file, 
                           nchannels=n_channels, 
                           nilluminations=1, 
                           nangles=n_angles,
                           ntiles= n_z)

    for n, tiff_dir in enumerate(dirs):
        print(tiff_dir)
        view_details = os.path.basename(tiff_dir).split("_")
        angle = -OPM_ANGLE if "view1" in view_details[-1] else OPM_ANGLE
        angle_i = angle_idxc[n]
        position_str = position
        position_i = int(position_str[1:])
        time_str = time_strs[n]
        time_i = int(time_str)
        z_str = z_strs[n]
        z_i = int(z_str)
        channel_str = channels_strs[n]
        channel_i = int(channel_str)

        print("angle", angle)
        print("position", position_str)
        print("time", time_str)
        print("z", z_str)
        print("channel", channel_str)

        # print(stack_dir)
        print(os.path.join(tiff_dir, "HamamatsuHam_DCAM-dopm/*.tif"))
        stacks = glob(os.path.join(tiff_dir, "HamamatsuHam_DCAM-dopm/*.tif"))
        print("tifs in dir", stacks)
        print("reading ome of first tif", stacks[0])
        # get ome metadata using ome-types library, tifffile doesnt parse xml into a dict unlike mm metadata
        omexml = from_tiff(stacks[0])
        iminfo = omexml.images[0]

        size_x = iminfo.pixels.size_x
        size_y = iminfo.pixels.size_y
        size_z = len(omexml.images)
        vox_size_x_um = iminfo.pixels.physical_size_x
        vox_size_y_um = iminfo.pixels.physical_size_y
        vox_size_z_um = iminfo.pixels.physical_size_z
        
        print("size_x", size_x)
        print("size_y", size_y)
        print("size_z", size_z)

        imstack = np.zeros((size_z, size_y, size_x), dtype=np.uint16)

        start_time = timer()

        # read the mm-split 4gb tiff stacks into one big stack (matrix), think about memory concerns...
        
        offset = 0
        for i in range(len(stacks)):
            print("loading",  stacks[i])
            with TiffFile(stacks[i]) as tif:
                mm_metadata = tif.micromanager_metadata
                for n, page in enumerate(tif.pages):
                    # print("page", n)
                    imstack[n+offset,:,:] = page.asarray() # expects zyx
            offset += n+1

        end_time = timer()
        print("Read time", (end_time-start_time), "ms")

        laser_meta = mm_metadata['Summary']['UserData']['laser']
        filter_meta = mm_metadata['Summary']['UserData']['filter']
        
        channel_attr[channel_i] = f"{laser_meta}"
        angle_attr[angle_i] = f"{angle}"

        rotation_matrix = affine.rotation_matrix(theta=np.pi/2, axis='x', square=False)
        skew_angle = np.pi*(angle-OPM_ANGLE)/180
        scale_matrix = affine.scale_matrix(sz=np.sin(skew_angle), square=False)
        skew_matrix = affine.skew_matrix(skew_angle, square=False)
        total_affine = skew_matrix @ scale_matrix @ rotation_matrix

        start_time = timer()  # start timer for saving h5 file

        angle_from_zero = angle+OPM_ANGLE
        print("angle", angle_i, "time",time_i,"tile",z_i,"channel",channel_i)
        bdv_writer.append_view(
            imstack, angle=angle_i, time=time_i, tile=z_i, channel=channel_i,
            calibration=(vox_size_x_um, vox_size_y_um, vox_size_z_um),
            voxel_size_xyz=(vox_size_x_um, vox_size_y_um, vox_size_z_um),
            voxel_units='micron',
            m_affine=total_affine)
        print(f"dataset in {save_file}")
        bdv_writer.write_xml()
        
    # 
    bdv_writer.set_attribute_labels('angle', angle_attr) 
    bdv_writer.set_attribute_labels('channel', channel_attr)
    """
    bdv_writer.append_affine(
        affine.rotation_matrix(theta=np.pi/2, axis='x', square=False),
        name_affine='90 deg rotation about x')
    skew_angle = -np.pi*(angle)/180
    bdv_writer.append_affine(
        affine.scale_matrix(sz=np.sin(skew_angle), square=False),
        name_affine='scale before shear')
    bdv_writer.append_affine(
        affine.skew_matrix(skew_angle, square=False),
        name_affine='shear to deskew')
    bdv_writer.set_attribute_labels('angle', angle_attr) 
    bdv_writer.set_attribute_labels('channel', channel_attr)
    """
    bdv_writer.close()

20250124-122939
channels ['0000', '0000'] time ['0000', '0000'] z ['0000', '0000'] angle [0, 90]
n_channels 1 n_times 1 n_z 1 n_angles 2
E:/dopm_reslicing/y_stage_scan_20241216-171205_c0p0\dOPM_t0000_p0000_z0000_c0000_view1
angle -45
position p0000
time 0000
z 0000
channel 0000
E:/dopm_reslicing/y_stage_scan_20241216-171205_c0p0\dOPM_t0000_p0000_z0000_c0000_view1\HamamatsuHam_DCAM-dopm/*.tif
tifs in dir ['E:/dopm_reslicing/y_stage_scan_20241216-171205_c0p0\\dOPM_t0000_p0000_z0000_c0000_view1\\HamamatsuHam_DCAM-dopm\\HamamatsuHam_DCAM-dopm_MMStack.ome.tif', 'E:/dopm_reslicing/y_stage_scan_20241216-171205_c0p0\\dOPM_t0000_p0000_z0000_c0000_view1\\HamamatsuHam_DCAM-dopm\\HamamatsuHam_DCAM-dopm_MMStack_1.ome.tif', 'E:/dopm_reslicing/y_stage_scan_20241216-171205_c0p0\\dOPM_t0000_p0000_z0000_c0000_view1\\HamamatsuHam_DCAM-dopm\\HamamatsuHam_DCAM-dopm_MMStack_2.ome.tif', 'E:/dopm_reslicing/y_stage_scan_20241216-171205_c0p0\\dOPM_t0000_p0000_z0000_c0000_view1\\HamamatsuHam_DCAM-dopm\\Hamamatsu

IndexError: index out of range